In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# How to use this notebook
[Google Cloud AI Platform Notebooks](https://cloud.google.com/ml-engine/docs/notebooks/overview) is a hosted JupyterLab environment that comes optimized for machine learning.  
  
### Instructions:
1. [Set up your Google Cloud Platform (GCP) project](https://console.cloud.google.com/cloud-resource-manager?_ga=2.150499254.-1267767919.1550615012).
2. [Enable billing for the GCP project.](https://cloud.google.com/billing/docs/how-to/modify-project)
3. [Enable the Compute Engine API.](https://console.cloud.google.com/flows/enableapi?apiid=compute.googleapis.com&_ga=2.150499254.-1267767919.1550615012)
4. [Create a new AI Platform Notebooks instance.](https://cloud.google.com/ml-engine/docs/notebooks/create-new)
    - Select "TensorFlow 1.x" as the instance type, or ML framework.
    - Including a GPU for this tutorial is not necessary. However, it may be helpful for future (or existing SageMaker) models that do require GPUs. [TODO: add support for using GPU]
5. Select "Open Jupyterlab" for thte 

# Preprocess MNIST dataset
Convert the MNIST images into TFRecords and upload the TFRecords to Google Cloud Storage (GCS).
  
## Install libraries
AI Platform Notebooks comes pre-installed with TensorFlow and Keras. 

In [16]:
import os
import numpy as np
from tensorflow.keras.datasets import mnist
import tensorflow as tf

In [48]:
print(tf.__version__)

1.14.0


## Set up Google Cloud Storage
When working with AI Platform, it is recommended to store TFRecords in GCS. More information on working with GCS with AI Platform can be found [here.](https://cloud.google.com/ml-engine/docs/tensorflow/working-with-cloud-storage) 
   
Specify a name for your existing (or new) GCS bucket with the BUCKET_NAME. It should be prefixed with "gs://" and must be unique across all buckets in Cloud Storage.

In [36]:
BUCKET_NAME='gs://ml-model-migration'
REGION='us-central1'

If the GCS bucket must be created, run the following bash command. Creating a GCS bucket can either be done through the front-end or command line. More instructions on creating a Google Cloud Storage Bucket can be found [here.](https://cloud.google.com/storage/docs/creating-buckets)

In [37]:
!gsutil mb -l {REGION} {BUCKET_NAME}

Creating gs://ml-model-migration/...


The AI Platform notebook is authenticated as the default Compute Engine service account (unless otherwise specified at the time of notebook creation). This means that it should already have authorization to create new buckets and read/write from existing buckets. 
  
If you are getting authorization errors, review the relevant service account's IAM permissions. If the storage bucket is not part of the same project as this Notebook, the Compute Engine service account may need to be granted access to the Cloud Storage bucket.  
  
To check which service account should be granted access, verify which service account is authenticated for this notebook. The service account should be included as the "email" field for the access token's info:

In [55]:
import subprocess

def access_token():
    return subprocess.check_output(
        'gcloud auth application-default print-access-token',
        shell=True,
    ).decode().strip()

!curl https://www.googleapis.com/oauth2/v1/tokeninfo?access_token={access_token()}

{
  "issued_to": "111616252376478783342",
  "audience": "111616252376478783342",
  "scope": "https://www.googleapis.com/auth/userinfo.email https://www.googleapis.com/auth/cloud-platform",
  "expires_in": 3138,
  "email": "946556229441-compute@developer.gserviceaccount.com",
  "verified_email": true,
  "access_type": "offline"
}


In addition to the [Google Cloud Storage Python Client](https://github.com/googleapis/google-cloud-python/tree/master/storage), some Python modules support reading/writing files locally and with GCS interchangeably. The module will read/write from GCS if the "gs://" prefix for the file or directory is specified.   
  
Options include:
- [tf.io.gfile](https://www.tensorflow.org/api_docs/python/tf/io/gfile) for file I/O wrappers without thread locking
- [tf.io.TFRecordWriter](https://www.tensorflow.org/api_docs/python/tf/io/TFRecordWriter) for writing records to a TFRecords file in GCS
- [pandas 0.24.0 or later](https://pandas.pydata.org/)

There should be no differences between Google Cloud AI Platform and AWS Sagemaker for the below code.

In [18]:
def load_mnist_data():   
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = np.reshape(x_train, [-1, 28,28,1])
    x_test = np.reshape(x_test, [-1, 28,28,1])
    train_data = {'images':x_train, 'labels':y_train}
    test_data = {'images':x_test, 'labels':y_test}
    return train_data, test_data

In [39]:
def export_tfrecords(data_set, name, directory):
    """Converts MNIST dataset to tfrecords.
    
    Args:
        data_set: Dictionary containing a numpy array of images and labels.
        name: Name given to the exported tfrecord dataset.
        directory: Directory that the tfrecord files will be saved in.
    """
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
    images = data_set['images']
    labels = data_set['labels']
    num_examples = images.shape[0]  
    rows = images.shape[1]
    cols = images.shape[2]
    depth = images.shape[3]

    filename = os.path.join(directory, name + '.tfrecords')
    print('Writing', filename)
   
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())
    writer.close()

In [19]:
train_data, test_data = load_mnist_data()

In [40]:
export_tfrecords(train_data, "mnist_train", BUCKET_NAME)

Writing gs://ml-model-migration/mnist_train.tfrecords


In [14]:
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  